# Data Cleaning
___

After running the code in `webscraping.py`, we now have a raw `.csv` file representing the stats for every NFL player since 1970 (courtesy of [PFR](https://www.pro-football-reference.com/)). We should have the data we need to make predictions about a player's future production. In this notebook, we'll do some basic data cleaning and inspection.

### Contents:
1. [Drop unwanted columns](#unwanted)
1. [Handle missing data](#missing)
1. [Check for proper datatypes](#datatypes)
1. [Dummify features](#dummify)
1. [Create columns for next year's fantasy production](#next_year)
1. [Export clean data](#export)

### Import libraries
___

In [2]:
import numpy as np
import os
import pandas as pd

from math import floor

### Load the data
___

In [3]:
df = pd.read_csv('../data/fantasy.csv')
df.head()

,player,team,fantasy_pos,age,g,gs,pass_cmp,pass_att,pass_yds,pass_td,...,two_pt_md,two_pt_pass,fantasy_points,fantasy_points_ppr,draftkings_points,fanduel_points,vbd,fantasy_rank_pos,fantasy_rank_overall,year
0,Christian McCaffrey,CAR,RB,23,16.0,16.0,0.0,2.0,0.0,0.0,...,1.0,NaN,355.0,471.2,477.2,413.2,215.0,1,1.0,2019
1,Lamar Jackson,BAL,QB,22,15.0,15.0,265.0,401.0,3127.0,36.0,...,NaN,NaN,416.0,415.7,429.7,421.7,152.0,1,2.0,2019
2,Derrick Henry,TEN,RB,25,15.0,15.0,0.0,0.0,0.0,0.0,...,NaN,NaN,277.0,294.6,303.6,285.6,136.0,2,3.0,2019
3,Aaron Jones,GNB,RB,25,16.0,16.0,0.0,0.0,0.0,0.0,...,NaN,NaN,266.0,314.8,322.8,290.3,125.0,3,4.0,2019
4,Ezekiel Elliott,DAL,RB,24,16.0,16.0,0.0,0.0,0.0,0.0,...,NaN,NaN,258.0,311.7,319.7,284.7,117.0,4,5.0,2019


In [4]:
df.shape

(26128, 33)

# Drop unwanted columns<a id="unwanted"></a>
___

We have over 25,000 rows and 33 columns total, so in an effort to make our dataframe more manageable, we'll drop the columns we have no intention of using.

In [5]:
df.columns

Index(['player', 'team', 'fantasy_pos', 'age', 'g', 'gs', 'pass_cmp',
       'pass_att', 'pass_yds', 'pass_td', 'pass_int', 'rush_att', 'rush_yds',
       'rush_yds_per_att', 'rush_td', 'targets', 'rec', 'rec_yds',
       'rec_yds_per_rec', 'rec_td', 'fumbles', 'fumbles_lost', 'all_td',
       'two_pt_md', 'two_pt_pass', 'fantasy_points', 'fantasy_points_ppr',
       'draftkings_points', 'fanduel_points', 'vbd', 'fantasy_rank_pos',
       'fantasy_rank_overall', 'year'],
      dtype='object')

In [6]:
# These columns are all alternative scoring metrics.
# They aren't what we aim to predict, and including them in a model would introduce data leakage.
df.drop(columns=['draftkings_points', 'fanduel_points', 'vbd', 'fantasy_rank_pos', 'fantasy_rank_overall'],
        inplace=True)

# Handle missing data<a id="missing"></a>
---

Here, we'll check for missing values and handle them appropriately.

In [7]:
df.isnull().sum()

player                    0
team                      0
fantasy_pos            2984
age                       0
g                        98
gs                      241
pass_cmp                  9
pass_att                  9
pass_yds                  9
pass_td                   9
pass_int                 10
rush_att                  9
rush_yds                  9
rush_yds_per_att      11896
rush_td                   9
targets               11346
rec                       9
rec_yds                   9
rec_yds_per_rec        7301
rec_td                    9
fumbles                  10
fumbles_lost          18287
all_td                    0
two_pt_md             25278
two_pt_pass           25676
fantasy_points         3604
fantasy_points_ppr     2874
year                      0
dtype: int64

Since we have so many missing values, we'll investigate the columns with missing data one by one.

### `fantasy_pos`<a id="fantasy_pos"></a>
___

In [7]:
df.loc[df['fantasy_pos'].isnull(), :].head()

,player,team,fantasy_pos,age,g,gs,pass_cmp,pass_att,pass_yds,pass_td,...,rec_yds_per_rec,rec_td,fumbles,fumbles_lost,all_td,two_pt_md,two_pt_pass,fantasy_points,fantasy_points_ppr,year
77,Michael Burton,WAS,NaN,27,10.0,2.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,2019
536,Jerell Adams,HOU,NaN,27,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,2019
537,Stephen Anderson,LAC,NaN,26,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,2019
538,Evan Baylis,GNB,NaN,26,5.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,2019
539,Kendall Blanton,LAR,NaN,24,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,2019


In [8]:
# Examine one player with missing fantasy_pos to see what's up.
df.loc[df['player'] == 'Michael Burton', :]

,player,team,fantasy_pos,age,g,gs,pass_cmp,pass_att,pass_yds,pass_td,...,rec_yds_per_rec,rec_td,fumbles,fumbles_lost,all_td,two_pt_md,two_pt_pass,fantasy_points,fantasy_points_ppr,year
77,Michael Burton,WAS,NaN,27,10.0,2.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,2019
1124,Michael Burton,CHI,RB,26,8.0,1.0,0.0,0.0,0.0,0.0,...,6.0,0.0,0.0,0.0,0,NaN,NaN,1.0,1.6,2018
1727,Michael Burton,CHI,RB,25,16.0,3.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0,NaN,NaN,2.0,3.7,2017
2344,Michael Burton,DET,NaN,24,15.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,2016
2807,Michael Burton,DET,TE,23,16.0,7.0,0.0,0.0,0.0,0.0,...,6.5,1.0,0.0,0.0,1,NaN,NaN,10.0,16.1,2015


It looks like players with a missing `fantasy_pos` for one year may have their position listed for a different year. Wherever we see a player with a missing position, let's enter the position the player was most recently listed as.

In [9]:
def get_position(player):
    # Get all positions the player has been listed as.
    positions = df.loc[df['player'] == player, 'fantasy_pos'].values
    positions = [pos for pos in positions if str(pos) != 'nan']
    
    try:
        # Return the most recent position the player has been listed as, if any.
        return positions[0]
    except IndexError:
        return None

In [10]:
# Get all players with missing fantasy_pos and the corresponding year.
players_missing_position = df.loc[df['fantasy_pos'].isnull(), ['player', 'year']].values

for player, year in players_missing_position:
    position = get_position(player)
    
    # Check if get_position() returned a valid position
    if position:
        df.loc[(df['player'] == player) & (df['year'] == year), 'fantasy_pos'] = position

In [11]:
# Check to see if there are any players with missing positions left.
df.loc[df['fantasy_pos'].isnull(), :].head()

,player,team,fantasy_pos,age,g,gs,pass_cmp,pass_att,pass_yds,pass_td,...,rec_yds_per_rec,rec_td,fumbles,fumbles_lost,all_td,two_pt_md,two_pt_pass,fantasy_points,fantasy_points_ppr,year
538,Evan Baylis,GNB,NaN,26,5.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,2019
539,Kendall Blanton,LAR,NaN,24,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,2019
541,Deante Burton,DAL,NaN,25,3.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,2019
544,Jordan Chunn,DAL,NaN,24,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,2019
545,Chandler Cox,MIA,NaN,23,13.0,3.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,2019


In [12]:
# Check how many players with missing positions are left
df.loc[df['fantasy_pos'].isnull(), :].shape

(1048, 28)

It's not quite clear why some of these players have no listed position, and since there are over 1000 of them, we can't manually enter their positions as listed on the [PFR player pages](https://www.pro-football-reference.com/players/B/BaylEv00.htm). None of them seem to be particularly productive, so for the sake of simplicity, we'll drop the players who have no position listed for any season.

In [13]:
missing_pos_ix = df.loc[df['fantasy_pos'].isnull(), :].index
df.drop(index=missing_pos_ix, inplace=True)

### `g`<a id="g"></a>
---

In [14]:
df.loc[df['g'].isnull(), :].head()

,player,team,fantasy_pos,age,g,gs,pass_cmp,pass_att,pass_yds,pass_td,...,rec_yds_per_rec,rec_td,fumbles,fumbles_lost,all_td,two_pt_md,two_pt_pass,fantasy_points,fantasy_points_ppr,year
4822,Adrian Arrington,NOR,WR,27,NaN,NaN,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,2012
4824,Edwin Baker,SDG,RB,21,NaN,NaN,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,2012
4825,Jake Ballard,NWE,TE,25,NaN,NaN,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,2012
4826,Joe Banyard,MIN,RB,24,NaN,NaN,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,2012
4832,Jahvid Best,DET,RB,23,NaN,NaN,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,2012


The PFR player pages for [Adrian Arrington](https://www.pro-football-reference.com/players/A/ArriAd00.htm), [Edwin Baker](https://www.pro-football-reference.com/players/B/BakeEd01.htm), and [Jake Ballard](https://www.pro-football-reference.com/players/B/BallJa21.htm) seem to indicate that these players didn't see the field on the year listed above, because of injury or other reason. We'll make the assumption that any player with a null value of `g` in a given season did not play, and simply impute zero.

In [15]:
df.loc[df['g'].isnull(), 'g'] = 0

### `gs`
---

In [16]:
df.loc[df['gs'].isnull(), :].head()

,player,team,fantasy_pos,age,g,gs,pass_cmp,pass_att,pass_yds,pass_td,...,rec_yds_per_rec,rec_td,fumbles,fumbles_lost,all_td,two_pt_md,two_pt_pass,fantasy_points,fantasy_points_ppr,year
3066,Bradie Ewing,JAX,RB,25,0.0,NaN,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,2014
3502,Jared Abbrederis,GNB,WR,24,0.0,NaN,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,2014
3507,Vick Ballard,IND,RB,24,0.0,NaN,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,2014
3510,Kenjon Barner,PHI,RB,24,0.0,NaN,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,2014
3515,Kapri Bibbs,DEN,RB,21,0.0,NaN,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,2014


In [17]:
# Check if any player with no starts has ever played a game
df.loc[(df['gs'].isnull()) & (df['g'] > 0), :].head()

,player,team,fantasy_pos,age,g,gs,pass_cmp,pass_att,pass_yds,pass_td,...,rec_yds_per_rec,rec_td,fumbles,fumbles_lost,all_td,two_pt_md,two_pt_pass,fantasy_points,fantasy_points_ppr,year


As with the missing `g` entries, we'll assume a missing `gs` entry means the player had no starts for the given season. 

In [18]:
df.loc[df['gs'].isnull(), 'gs'] = 0

### `pass_cmp`, `pass_att`, `pass_yds`, `pass_td`, and `pass_int`
---

In [19]:
missing_pass_stats = (df['pass_cmp'].isnull()
                      | df['pass_att'].isnull()
                      | df['pass_yds'].isnull()
                      | df['pass_td'].isnull()
                      | df['pass_int'].isnull())

In [20]:
df.loc[missing_pass_stats, ['player', 'fantasy_pos', 'year']
                            + [col for col in df.columns if col.startswith('pass')]]

,player,fantasy_pos,year,pass_cmp,pass_att,pass_yds,pass_td,pass_int
4854,Mike Cox,RB,2012,0.0,0.0,0.0,0.0,NaN
4877,Daniel Graham,TE,2012,NaN,NaN,NaN,NaN,NaN
4955,Stephen Spach,TE,2012,NaN,NaN,NaN,NaN,NaN
4959,Maurice Stovall,WR,2012,NaN,NaN,NaN,NaN,NaN
5512,Eldra Buckley,RB,2011,NaN,NaN,NaN,NaN,NaN
5544,David Martin,TE,2011,NaN,NaN,NaN,NaN,NaN
5546,Mike McNeill,TE,2011,NaN,NaN,NaN,NaN,NaN
5567,Jordan Todman,RB,2011,NaN,NaN,NaN,NaN,NaN
6095,Victor Cruz,WR,2010,NaN,NaN,NaN,NaN,NaN


As all of these players are not quarterbacks, it's highly likely the missing passing statistics should be imputed with zero.

In [21]:
df.loc[missing_pass_stats, df.columns.str.startswith('pass')] = 0

### `rush_att`, `rush_yds`, and `rush_td`
---

In [22]:
missing_rush_stats = (df['rush_att'].isnull()
                      | df['rush_yds'].isnull()
                      | df['rush_td'].isnull())

In [23]:
df.loc[missing_rush_stats, ['player', 'fantasy_pos', 'year', 'rush_att', 'rush_yds', 'rush_td']]

,player,fantasy_pos,year,rush_att,rush_yds,rush_td
4877,Daniel Graham,TE,2012,NaN,NaN,NaN
4955,Stephen Spach,TE,2012,NaN,NaN,NaN
4959,Maurice Stovall,WR,2012,NaN,NaN,NaN
5512,Eldra Buckley,RB,2011,NaN,NaN,NaN
5544,David Martin,TE,2011,NaN,NaN,NaN
5546,Mike McNeill,TE,2011,NaN,NaN,NaN
5567,Jordan Todman,RB,2011,NaN,NaN,NaN
6095,Victor Cruz,WR,2010,NaN,NaN,NaN


We can see that six of the eight players with missing rushing statistics are not running backs, meaning they likely don't have rushing statistics for the years listed. Regarding [Eldra Buckley](https://www.pro-football-reference.com/players/B/BuckEl00.htm) and [Jordan Todman](https://www.pro-football-reference.com/players/T/TodmJo00.htm), neither appear to have rushed the ball in the seasons above.

In [24]:
df.loc[missing_pass_stats, df.columns.str.startswith('rush')] = 0

### `rush_yds_per_att`
---

It's possible that a null value for `rush_yds_per_att` indicates a player doesn't have any rushing attempts, since

$$rush\_yds\_per\_att = \frac{rush\_yds}{rush\_att}$$

In [25]:
df.loc[df['rush_yds_per_att'].isnull(), ['player', 'fantasy_pos', 'year', 'rush_yds_per_att', 'rush_yds', 'rush_att']].head()

,player,fantasy_pos,year,rush_yds_per_att,rush_yds,rush_att
12,Mark Andrews,TE,2019,NaN,0.0,0.0
19,Kenny Golladay,WR,2019,NaN,0.0,0.0
21,Zach Ertz,TE,2019,NaN,0.0,0.0
24,Jared Cook,TE,2019,NaN,0.0,0.0
27,DeVante Parker,WR,2019,NaN,0.0,0.0


In [26]:
# Show players with null rush_yds_per_att but at least 1 rush_att.
df.loc[(df['rush_yds_per_att'].isnull()) & (df['rush_att'] != 0), :]

,player,team,fantasy_pos,age,g,gs,pass_cmp,pass_att,pass_yds,pass_td,...,rec_yds_per_rec,rec_td,fumbles,fumbles_lost,all_td,two_pt_md,two_pt_pass,fantasy_points,fantasy_points_ppr,year


We can see that every player with a missing `rush_yds_per_att` entry has `rush_att = 0`. We could impute these to be zero, but that would be mathematically misleading. Instead, since we probably won't use the column for modeling, we'll just drop it all together.

In [27]:
df.drop(columns='rush_yds_per_att', inplace=True)

### `rec`, `rec_yds`, `rec_td`
---

In [28]:
missing_rec_stats = (df['rec'].isnull()
                     | df['rec_yds'].isnull()
                     | df['rec_td'].isnull())

In [29]:
df.loc[missing_rec_stats, ['player', 'fantasy_pos', 'year', 'rec', 'rec_yds', 'rec_td']]

,player,fantasy_pos,year,rec,rec_yds,rec_td
4877,Daniel Graham,TE,2012,NaN,NaN,NaN
4955,Stephen Spach,TE,2012,NaN,NaN,NaN
4959,Maurice Stovall,WR,2012,NaN,NaN,NaN
5512,Eldra Buckley,RB,2011,NaN,NaN,NaN
5544,David Martin,TE,2011,NaN,NaN,NaN
5546,Mike McNeill,TE,2011,NaN,NaN,NaN
5567,Jordan Todman,RB,2011,NaN,NaN,NaN
6095,Victor Cruz,WR,2010,NaN,NaN,NaN


Player pages for [Daniel Graham](https://www.pro-football-reference.com/players/G/GrahDa00.htm) and [Stephen Spach](https://www.pro-football-reference.com/players/S/SpacSt00.htm) indicate that they didn't have any receptions for the 2012 season. We'll take null values here to mean no receptions for the given season.

In [30]:
# Impute missing rec stats with 0.
df.loc[missing_rec_stats, ['rec', 'rec_yds', 'rec_td']] = 0

### `targets`
---

Per our data dictionary, we know that `targets` (the amount of times a player was thrown to) were not tracked until the 1992 season. We should expect that seasons before 1992 don't have the target data included.

In [31]:
df.loc[df['targets'].isnull(), ['player', 'fantasy_pos', 'year', 'targets', 'rec']].head()

,player,fantasy_pos,year,targets,rec
3066,Bradie Ewing,RB,2014,NaN,0.0
3069,Richie Brockel,TE,2014,NaN,0.0
3495,Aldrick Robinson,WR,2014,NaN,1.0
3502,Jared Abbrederis,WR,2014,NaN,0.0
3507,Vick Ballard,RB,2014,NaN,0.0


Unfortunately, we do see some players with a null entry for `targets` even after the 1992 season. We can safely assume the players with `rec = 0` don't have any targets for the seasons above, but [Aldrick Robinson](https://www.pro-football-reference.com/players/R/RobiAl00.htm) has `rec = 1` for the season listed, meaning he had to have been thrown the ball at least once. We'll impute missing target entries with the number of receptions (the number of receptions is always a strict lower bound for the number of targets).

In [32]:
df.loc[df['targets'].isnull(), 'targets'] = df.loc[df['targets'].isnull(), 'rec']

### `rec_yds_per_rec`
---

As with `rush_yds_per_att`, it seems likely that a null value for `rec_yds_per_rec` indicates a lack of receptions for the season.

In [33]:
df.loc[df['rec_yds_per_rec'].isnull(), ['player', 'fantasy_pos', 'year', 'rec_yds_per_rec', 'rec_yds', 'rec']].head()

,player,fantasy_pos,year,rec_yds_per_rec,rec_yds,rec
1,Lamar Jackson,QB,2019,NaN,0.0,0.0
11,Dak Prescott,QB,2019,NaN,0.0,0.0
13,Russell Wilson,QB,2019,NaN,0.0,0.0
29,Jameis Winston,QB,2019,NaN,0.0,0.0
43,Josh Allen,QB,2019,NaN,0.0,0.0


In [34]:
# Show players with null rec_yds_per_rec but at least 1 rec.
df.loc[(df['rec_yds_per_rec'].isnull()) & (df['rec'] != 0), :]

,player,team,fantasy_pos,age,g,gs,pass_cmp,pass_att,pass_yds,pass_td,...,rec_yds_per_rec,rec_td,fumbles,fumbles_lost,all_td,two_pt_md,two_pt_pass,fantasy_points,fantasy_points_ppr,year


We'll delete this column as we did with `rush_yds_per_att`.

In [35]:
df.drop(columns='rec_yds_per_rec', inplace=True)

### `fumbles`
---

In [36]:
df.loc[df['fumbles'].isnull(), :]

,player,team,fantasy_pos,age,g,gs,pass_cmp,pass_att,pass_yds,pass_td,...,rec_yds,rec_td,fumbles,fumbles_lost,all_td,two_pt_md,two_pt_pass,fantasy_points,fantasy_points_ppr,year
4854,Mike Cox,ATL,RB,27,8.0,7.0,0.0,0.0,0.0,0.0,...,22.0,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,2012
4877,Daniel Graham,NOR,TE,34,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,2012
4955,Stephen Spach,JAX,TE,30,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,2012
4959,Maurice Stovall,JAX,WR,27,10.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,0,NaN,NaN,NaN,NaN,2012
5512,Eldra Buckley,DET,RB,26,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,2011
5544,David Martin,BUF,TE,32,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,2011
5546,Mike McNeill,IND,TE,23,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,2011
5567,Jordan Todman,SDG,RB,21,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,2011
6095,Victor Cruz,NYG,WR,24,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,2010


Since most of these players have few to no game starts, we'll assume the missing `fumbles` entries should be zero.

In [37]:
# Impute with 0.
df.loc[df['fumbles'].isnull(), 'fumbles'] = 0

### `fumbles_lost`
---

In [38]:
df.loc[df['fumbles_lost'].isnull(), ['player', 'fantasy_pos', 'year', 'fumbles', 'fumbles_lost']].head()

,player,fantasy_pos,year,fumbles,fumbles_lost
3066,Bradie Ewing,RB,2014,0.0,NaN
3502,Jared Abbrederis,WR,2014,0.0,NaN
3507,Vick Ballard,RB,2014,0.0,NaN
3510,Kenjon Barner,RB,2014,0.0,NaN
3515,Kapri Bibbs,RB,2014,0.0,NaN


In [39]:
# Show players with missing fumbles_lost but > 0 fumbles
df.loc[(df['fumbles_lost'].isnull()) & (df['fumbles'] > 0), ['player', 'fantasy_pos', 'year', 'fumbles', 'fumbles_lost']].head()

,player,fantasy_pos,year,fumbles,fumbles_lost
4989,Aaron Rodgers,QB,2011,5.0,NaN
5002,Wes Welker,WR,2011,1.0,NaN
5019,Adrian Peterson,RB,2011,1.0,NaN
5022,Darren Sproles,RB,2011,1.0,NaN
5023,A.J. Green,WR,2011,1.0,NaN


To impute the missing `fumbles_lost` entries, we'll multiply the number of fumbles by the percentage of times a player who fumbles also loses the fumble:

$$fumbles\_lost = fumbles \times fumble\_pct$$

In [40]:
# Find out how often a player loses a fumble given that they fumbled.
n_fumbles = sum(df.loc[~(df['fumbles_lost'].isnull()), 'fumbles'].values)
n_fumbles_lost = sum(df.loc[~(df['fumbles_lost'].isnull()), 'fumbles_lost'].values)
fumble_pct = n_fumbles_lost / n_fumbles

In [41]:
# fumbles_lost = fumbles * fumble_pct
df.loc[df['fumbles_lost'].isnull(), 'fumbles_lost'] = (df.loc[df['fumbles_lost'].isnull(), 'fumbles'] * fumble_pct).map(floor)

### `two_pt_md` and `two_pt_pass`
---

There are over 25,000 playes with either a missing `two_pt_md` or `two_pt_pass` entry, so we'll just impute any missing value with zero.

In [42]:
# Impute missing `two_pt_md` with 0.
df['two_pt_md'].fillna(0, inplace=True)

In [43]:
# Impute missing `two_pt_pass`` with 0
df['two_pt_pass'].fillna(0, inplace=True)

### `fantasy_points`
---

To fill in the `fantasy_points` score, we'll define a function that calculates points scored based on the traditional standard scoring rules:

_1 point per 25 yards passing, 4 points per passing touchdown, -2 points per interception thrown, 1 point per 10 yards rushing/receiving, 6 points per TD, 2 points per two-point conversion, -2 points per fumble lost_

In [44]:
def get_standard_scoring(player, year):
    # Define standard scoring rules.
    standard_scoring = {
        'pass_yds': (1 / 25),
        'pass_td': 4,
        'pass_int': -2,
        'rush_yds': (1 / 10),
        'rec_yds': (1 / 10),
        'all_td': 6,
        'two_pt_md': 2,
        'two_pt_pass': 2,
        'fumbles_lost': -2,
    }
    
    # Define dataframe with player's stats from the given year.
    player_df = df.loc[(df['player'] == player) & (df['year'] == year), :]
    
    return sum([player_df[stat] * stat_value for stat, stat_value in standard_scoring.items()])

In [45]:
df.loc[df['fantasy_points'].isnull(), 'fantasy_points'] = get_standard_scoring(df['player'], df['year'])

### `fantasy_points_ppr`
---

As with `fantasy_points`, we'll calculate `fantasy_points_ppr` using the traditional PPR scoring rules (standard scoring with one point per reception):

_1 point per 25 yards passing, 4 points per passing touchdown, -2 points per interception thrown, 1 point per reception, 1 point per 10 yards rushing/receiving, 6 points per TD, 2 points per two-point conversion, -2 points per fumble lost_

In [46]:
def get_ppr_scoring(player, year):
    # Define dataframe with player's stats from the given year.
    player_df = df.loc[(df['player'] == player) & (df['year'] == year), :]
    
    # PPR scoring is just standard scoring plus 1 point per reception
    return get_standard_scoring(player, year) + player_df['rec']

In [47]:
df.loc[df['fantasy_points_ppr'].isnull(), 'fantasy_points_ppr'] = get_ppr_scoring(df['player'], df['year'])

### Confirm all missing values have been dealt with
---

In [48]:
df.isnull().sum()

player                0
team                  0
fantasy_pos           0
age                   0
g                     0
gs                    0
pass_cmp              0
pass_att              0
pass_yds              0
pass_td               0
pass_int              0
rush_att              0
rush_yds              0
rush_td               0
targets               0
rec                   0
rec_yds               0
rec_td                0
fumbles               0
fumbles_lost          0
all_td                0
two_pt_md             0
two_pt_pass           0
fantasy_points        0
fantasy_points_ppr    0
year                  0
dtype: int64

# Check for proper datatypes<a id="datatypes"></a>
---

We need to ensure that the entries in each column are of the proper datatype (numerical columns are of type int or float, text columns are of type object).

In [49]:
df.dtypes

player                 object
team                   object
fantasy_pos            object
age                     int64
g                     float64
gs                    float64
pass_cmp              float64
pass_att              float64
pass_yds              float64
pass_td               float64
pass_int              float64
rush_att              float64
rush_yds              float64
rush_td               float64
targets               float64
rec                   float64
rec_yds               float64
rec_td                float64
fumbles               float64
fumbles_lost          float64
all_td                  int64
two_pt_md             float64
two_pt_pass           float64
fantasy_points        float64
fantasy_points_ppr    float64
year                    int64
dtype: object

Every numerical column is as it should be, but there are perhaps a few text columns that we would like to dummify for ease of use.

# Dummify features<a id="dummify"></a>
---

Code here heavily borrowed from [this Stack Overflow post](https://stackoverflow.com/questions/38541636/keep-other-variables-when-executing-get-dummies-in-pandas).

In [50]:
# Dummify fantasy_pos column.
dummies = pd.get_dummies(df, prefix='is', columns=['fantasy_pos'])
dummies = dummies[[col for col in dummies.columns if col.startswith('is')]]

In [51]:
# Concatenate dummies dataframe with original dataframe.
df = pd.concat([df, dummies], axis=1)

In [52]:
df.head()

,player,team,fantasy_pos,age,g,gs,pass_cmp,pass_att,pass_yds,pass_td,...,all_td,two_pt_md,two_pt_pass,fantasy_points,fantasy_points_ppr,year,is_QB,is_RB,is_TE,is_WR
0,Christian McCaffrey,CAR,RB,23,16.0,16.0,0.0,2.0,0.0,0.0,...,19,1.0,0.0,355.0,471.2,2019,0,1,0,0
1,Lamar Jackson,BAL,QB,22,15.0,15.0,265.0,401.0,3127.0,36.0,...,7,0.0,0.0,416.0,415.7,2019,1,0,0,0
2,Derrick Henry,TEN,RB,25,15.0,15.0,0.0,0.0,0.0,0.0,...,18,0.0,0.0,277.0,294.6,2019,0,1,0,0
3,Aaron Jones,GNB,RB,25,16.0,16.0,0.0,0.0,0.0,0.0,...,19,0.0,0.0,266.0,314.8,2019,0,1,0,0
4,Ezekiel Elliott,DAL,RB,24,16.0,16.0,0.0,0.0,0.0,0.0,...,14,0.0,0.0,258.0,311.7,2019,0,1,0,0


In [53]:
# Map new column names to lowercase letters
df.columns = df.columns.str.lower()

We would ideally also like to dummify the `team` column, but as there are 32 NFL teams, it would make our dataframe too large and unwieldy.

# Create columns for next year's fantasy production<a id="next_year"></a>
---

Since our ultimate goal is to predict a player's fantasy output in year $n + 1$ based on his stats in year $n$, we'll add two columns to the dataframe with his `fantasy_points` and `fantasy_points_ppr` for the following season. An entry of `NaN` will indicate that the player doesn't have fantasy statistics for the following season (he didn't play the following year, e.g. due to retirement).

In [54]:
# This function will get fantasy_points for the following season.
def get_next_year_standard(player_row):
    is_player = df['player'] == player_row['player']
    is_next_year = df['year'] == player_row['year'] + 1
    
    try:
        return df.loc[is_player & is_next_year, 'fantasy_points'].values[0]
    except IndexError:
        return np.nan

# This function will get fantasy_points_ppr for the following season.
def get_next_year_ppr(player_row):
    is_player = df['player'] == player_row['player']
    is_next_year = df['year'] == player_row['year'] + 1
    
    try:
        return df.loc[is_player & is_next_year, 'fantasy_points_ppr'].values[0]
    except IndexError:
        return np.nan

# Create new columns for next year stats.
df['fantasy_points_next_year'] = df.apply(get_next_year_standard, axis=1)
df['fantasy_points_ppr_next_year'] = df.apply(get_next_year_ppr, axis=1)

# Export clean data<a id="export"></a>
---

In [55]:
try:
    os.mkdir('../data')
except FileExistsError:
    pass

In [56]:
df.to_csv('../data/clean.csv', index=False)